# Переобученная модель
Стоит задача написать модель, которая доходит до хорших метрик, но быстро начинает переобучаться

Импорт необходимых библиотек

In [ ]:
import sys

import numpy as np
from PIL import Image
import tensorflow as tf
import torchvision as tv
from livelossplot import PlotLossesKeras

sys.path.append('../../')

from core.datasets import get_ds
from core.make_answer import make_ans_file

Создаем датасеты

In [45]:
# train_ds, val_ds, test_ds = get_ds()
!git pull -f

error: Pulling is not possible because you have unmerged files.
hint: Fix them up in the work tree, and then use 'git add/rm <file>'
hint: as appropriate to mark resolution and make a commit.
fatal: Exiting because of an unresolved conflict.


Пишем модель

In [ ]:
model_ex = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), input_shape=(32, 32, 3), activation='relu'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.AveragePooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.AveragePooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])
model_ex.summary()

Компиляция модели

In [ ]:
model_ex.compile(optimizer='adam', metrics=['accuracy'],
        loss=tf.keras.losses.SparseCategoricalCrossentropy())

Создаем чекпоинты

In [ ]:
checkpoint_filepath = '../checkpoints/checkpoint_ex.h5'
callback_ = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
)

Обучаем модель

In [ ]:
model_ex.fit(train_ds, validation_data=val_ds, epochs=100,
          callbacks=[PlotLossesKeras(), callback_], verbose=False)

Скачиваем модель из эпохи, которая показала лучшую метрику

In [ ]:
model_ex_best = tf.keras.models.load_model(checkpoint_filepath)
pred = model_ex_best.predict(test_ds / 255., verbose=False)

Формируем ответы в папку ans

In [ ]:
make_ans_file('models/answers/model_ex/checkpoint_ex.ans', pred)